In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from utils import *
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Load the data

In [ ]:
datapath = '/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/UCSD/DSC/DSC180/ClimateBench - Plus/ClimateBench-Plus/DKL Gaussian Process/data/processed_data/'
# datapath = 'G://My Drive//UCSD//DSC//DSC180//ClimateBench - Plus//ClimateBench-Plus//DKL Gaussian Process//data//processed_data//'
simulations = ['ssp126', 'ssp370', 'ssp585', 'hist-GHG', 'hist-aer']

In [ ]:
X_train = []
Y_train = []

for i, simu in enumerate(simulations):
    input_name = 'inputs_' + simu + '.nc'
    output_name = 'outputs_' + simu + '.nc'
    # Just load hist data in these cases 'hist-GHG' and 'hist-aer'
    if 'hist' in simu:
        # load inputs 
        input_xr = xr.open_dataset(datapath + input_name)
            
        # load outputs                                                             
        output_xr = xr.open_dataset(datapath + output_name).mean(dim='member')
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400, "pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])
    
    # Concatenate with historical data in the case of scenario 'ssp126', 'ssp370' and 'ssp585'
    else:
        # load inputs 
        input_xr = xr.open_mfdataset([datapath + 'inputs_historical.nc', datapath + input_name]).compute()
            
        # load outputs                                                             
        output_xr = xr.concat([xr.open_dataset(datapath + 'outputs_historical.nc').mean(dim='member'),
                               xr.open_dataset(datapath + output_name).mean(dim='member')],
                               dim='time').compute()
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,"pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])

    print(input_xr.dims, simu)

    # Append to list 
    X_train.append(input_xr)
    Y_train.append(output_xr)

Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp126
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp370
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp585
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-GHG
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-aer


## Normalize the data


In [ ]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}
len_historical = 165

for var in ['CO2', 'CH4', 'SO2', 'BC']:
    # To not take the historical data into account several time we have to slice the scenario datasets
    # and only keep the historical data once (in the first ssp index 0 in the simus list)
    array = np.concatenate([X_train[i][var].data for i in [0, 3, 4]] + 
                           [X_train[i][var].sel(time=slice(len_historical, None)).data for i in range(1, 3)])
    print((array.mean(), array.std()))
    meanstd_inputs[var] = (array.mean(), array.std())

(1074.172303244536, 1755.690699230666)
(0.1927369743762821, 0.18457590641432994)
(2.5623359997066755e-12, 2.250114566783271e-11)
(1.4947905009818064e-13, 1.0313342554838387e-12)


In [ ]:
# normalize input data 
X_train_norm = [] 
for i, train_xr in enumerate(X_train): 
    for var in ['CO2', 'CH4', 'SO2', 'BC']: 
        var_dims = train_xr[var].dims
        train_xr=train_xr.assign({var: (var_dims, normalize(train_xr[var].data, var, meanstd_inputs))}) 
    X_train_norm.append(train_xr)

In [ ]:
var_to_predict =  'tas'
# skip_historical set to (i < 2) because of the order of the scenario and historical runs in the X_train and Y_train lists.
# In details: ssp126 0, ssp370 1 = skip historical part of the data, ssp585 2, hist-GHG 3 and hist-aer 4 = keep the whole sequence
X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis = 0)
Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis=0)
# add a dimension to the output data
Y_train_all = Y_train_all[..., np.newaxis]


X_train_all = X_train_all.reshape(726, 4, 96, 144)
Y_train_all = Y_train_all.reshape(726, 1, 96, 144)
print(X_train_all.shape)
print(Y_train_all.shape)

(726, 4, 96, 144)
(726, 1, 96, 144)


## Setup Sparse Representation

Each datapoint will be represented with 2 tuples: (index, value) where the index in this case is the `latitude` and `longitude` and the value is the `value` of the pixel at that location.